In [98]:
import pandas as pd
import os

In [122]:
new_dfs = []

for file in os.listdir('demo'):
    if file.endswith('csv'):
        curr_year = int(file.split('.')[0].replace('ACSST5Y', ''))
        df = pd.read_csv(f'demo/{file}', header=[1])
        if 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American' not in set(df.columns):
            print('Column not found, adding columns')
            for column in df.columns:
                if column.startswith('Total!!Estimate!!'):
                    df[column.replace('Total!!Estimate!!', 'Estimate!!Total!!')] = df[column]
            for column in df.columns:
                if 'Estimate!!Total!!RACE AND HISPANIC OR LATINO ORIGIN' in column:
                    df[column.replace('Estimate!!Total!!RACE AND HISPANIC OR LATINO ORIGIN', 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN')] = df[column]
        if 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino' not in df.columns:
            try:
                df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'] = df['Estimate!!Total!!Total population!!White alone, not Hispanic or Latino']
            except:
                df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'] = df['Estimate!!Total!!White alone, not Hispanic or Latino']

        new_df = pd.DataFrame()
        new_df['tract'] = df['Geographic Area Name']
        new_df['race_percent_white'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'].replace('-', 0).astype(float)
        new_df['race_percent_black'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American'].replace('-', 0).astype(float)
        new_df['race_percent_native'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!American Indian and Alaska Native'].replace('-', 0).astype(float)
        new_df['race_percent_asian'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Asian'].replace('-', 0).astype(float)
        new_df['race_percent_pacific_islander'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Native Hawaiian and Other Pacific Islander'].replace('-', 0).astype(float)
        new_df['race_percent_hispanic'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White'].replace('-', 0).astype(float) - df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'].replace('-', 0).astype(float)
        new_df['race_percent_other'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Some other race'].replace('-', 0).astype(float)
        new_df['income_percent_below_15K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$1 to $9,999 or loss"].replace('-', 0).astype(float) +\
                                            df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$10,000 to $14,999"].replace('-', 0).astype(float)
        new_df['income_percent_15K_35K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$15,000 to $24,999"].replace('-', 0).astype(float) +\
                                        df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$25,000 to $34,999"].replace('-', 0).astype(float)
        new_df['income_percent_35K_65K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$35,000 to $49,999"].replace('-', 0).astype(float) +\
                                        df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$50,000 to $64,999"].replace('-', 0).astype(float)
        new_df['income_percent_above_65K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$65,000 to $74,999"].replace('-', 0).astype(float) +\
                                            df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$75,000 or more"].replace('-', 0).astype(float)
        
        new_df['year'] = curr_year
        new_dfs.append(new_df)

new_df = pd.concat(new_dfs).sort_values(['year', 'tract']).reset_index(drop=True)

for column in new_df.columns:
    if 'percent' in column:
        new_df[column] = new_df[column].div(100, axis=0)


Column not found, adding columns
Column not found, adding columns
Column not found, adding columns


In [123]:
new_df

,tract,race_percent_white,race_percent_black,race_percent_native,race_percent_asian,race_percent_pacific_islander,race_percent_hispanic,race_percent_other,income_percent_below_15K,income_percent_15K_35K,income_percent_35K_65K,income_percent_above_65K,year
0,"Census Tract 1005, Allegheny County, Pennsylvania",0.286,0.657,0.000,0.000,0.0,0.000,0.000,0.369,0.296,0.202,0.047,2014
1,"Census Tract 1011, Allegheny County, Pennsylvania",0.793,0.115,0.000,0.025,0.0,0.028,0.004,0.331,0.328,0.182,0.050,2014
2,"Census Tract 1014, Allegheny County, Pennsylvania",0.823,0.118,0.003,0.012,0.0,0.021,0.004,0.251,0.285,0.240,0.134,2014
3,"Census Tract 1016, Allegheny County, Pennsylvania",0.004,0.989,0.000,0.000,0.0,0.007,0.000,0.377,0.253,0.130,0.031,2014
4,"Census Tract 1017, Allegheny County, Pennsylvania",0.204,0.761,0.010,0.000,0.0,0.011,0.000,0.461,0.221,0.150,0.060,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,Census Tract 9810; Allegheny County; Pennsylvania,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,2022
3590,Census Tract 9811; Allegheny County; Pennsylvania,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,2022
3591,Census Tract 9812; Allegheny County; Pennsylvania,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,2022
3592,Census Tract 9818; Allegheny County; Pennsylvania,0.712,0.218,0.000,0.047,0.0,0.010,0.000,0.483,0.272,0.085,0.046,2022


In [124]:
new_df.to_csv('total_demographics.csv', index=False)